In [14]:
import pandas as pd
df = pd.read_csv("../assets/elaws_links.csv")
df.head(3)

,title,link_to_page,link_to_doc_file,asset_path
0,"Food Safety and Quality Act, 2001, S.O. 2001, ...",https://www.ontario.ca/laws/statute/01f20,https://www.ontario.ca/laws/docs/01f20_e.doc,assets/elaws_pdfs/01f20_e.pdf
1,"Nutrient Management Act, 2002, S.O. 2002, c. 4",https://www.ontario.ca/laws/statute/02n04,https://www.ontario.ca/laws/docs/02n04_e.doc,assets/elaws_pdfs/02n04_e.pdf
2,"Animal Health Act, 2009, S.O. 2009, c. 31",https://www.ontario.ca/laws/statute/09a31,https://www.ontario.ca/laws/docs/09a31_e.doc,assets/elaws_pdfs/09a31_e.pdf


In [11]:
# Setup a non-OCR document converter
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.datamodel.base_models import InputFormat
from docling.chunking import HybridChunker
from src.retrievers import make_text_chunk

# We don't need OCR for these PDFs
pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = False

converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

# Setup a hybrid chunker that respects headings and hierarchy
chunker = HybridChunker(max_tokens=1000)

In [15]:
# Process each PDF
from pathlib import Path

all_chunks = []
for idx, row in df.iterrows():
    try:
        print(f"Processing {row.title}")
        result = converter.convert(Path("../") / row.asset_path)
        chunk_iter = chunker.chunk(result.document)
        doc_chunks = [
            make_text_chunk(x, doc_uri=row.link_to_page) 
            for x in chunk_iter
            ]
        all_chunks.extend(doc_chunks)
    except Exception as e:
        print(f"Error processing {row.asset_path}: {str(e)}")

Processing Food Safety and Quality Act, 2001, S.O. 2001, c. 20
Processing Nutrient Management Act, 2002, S.O. 2002, c. 4
Processing Animal Health Act, 2009, S.O. 2009, c. 31
Processing Commodity Board Members Act, R.S.O. 1990, c. C.18
Processing Education Act, R.S.O. 1990, c. E.2
Processing Farm Products Payments Act, R.S.O. 1990, c. F.10
Processing Health Protection and Promotion Act, R.S.O. 1990, c. H.7
Processing Highway Traffic Act, R.S.O. 1990, c. H.8
Processing Milk Act, R.S.O. 1990, c. M.12
Processing Ministry of Agriculture, Food and Rural Affairs Act, R.S.O. 1990, c. M.16
Processing Retail Sales Tax Act, R.S.O. 1990, c. R.31
Processing Farming and Food Production Protection Act, 1998, S.O. 1998, c. 1
Processing Commodity Boards and Marketing Agencies Act, R.S.O. 1990, c. C.19


In [16]:
pd.DataFrame(all_chunks)

,doc_uri,pages,doc_refs,headings,captions,text,enriched_text
0,https://www.ontario.ca/laws/statute/01f20,[1],"[#/texts/1, #/texts/2, #/texts/3, #/texts/4, #...",[Français],None,1.\nPurposes\n2.\nDefinitions\n3.\nDirectors\n...,\n Headings:Français\n\n ...
1,https://www.ontario.ca/laws/statute/01f20,[1],"[#/texts/30, #/texts/31]","[Consolidation Period: From November 29, 2021 ...",None,"Last amendment: 2019, c. 15, Sched. 22, s. 93....",\n Headings:Consolidation Period: F...
2,https://www.ontario.ca/laws/statute/01f20,[1],"[#/texts/40, #/texts/41, #/texts/42, #/texts/4...",[INSPECTIONS AND ORDERS],None,INSPECTIONS\nInspectors\nSearch without warran...,\n Headings:INSPECTIONS AND ORDERS\...
3,https://www.ontario.ca/laws/statute/01f20,"[1, 2]","[#/texts/62, #/tables/0]",[ORDERS],None,"1\n34., 1 = Hearing by director. 34., 2 = . 35...",\n Headings:ORDERS\n\n C...
4,https://www.ontario.ca/laws/statute/01f20,[2],"[#/texts/68, #/texts/69, #/texts/70, #/texts/71]",[Purposes],None,"1 The purposes of this Act are to provide for,...",\n Headings:Purposes\n\n ...
...,...,...,...,...,...,...,...
7459,https://www.ontario.ca/laws/statute/90c19,[1],[#/texts/15],[Authority of commodity board or marketing age...,None,(3) Where the Lieutenant Governor in Council ...,\n Headings:Authority of commodity ...
7460,https://www.ontario.ca/laws/statute/90c19,[1],[#/texts/17],[Authority may be revoked],None,(4) The Lieutenant Governor in Council may b...,\n Headings:Authority may be revoke...
7461,https://www.ontario.ca/laws/statute/90c19,[1],[#/texts/19],[Regulations],None,3. The Lieutenant Governor in Council may make...,\n Headings:Regulations\n\n ...
7462,https://www.ontario.ca/laws/statute/90c19,[1],"[#/texts/21, #/texts/22]",[Offence],None,4. (1) Every person who fails to comply with ...,\n Headings:Offence\n\n ...
